Теперь с помощью LLM создадим датасет из запросов и ответов, чтобы по нему уже оценивать метрики системы поиска.

In [1]:
import numpy as np
import pandas as pd
import json
import regex
import torch
from torch import Tensor
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm.auto import tqdm

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
torch.manual_seed(142)
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else 'cpu'
print(device)

cuda


In [ ]:
model_name = 'HuggingFaceTB/SmolLM3-3B'
tokenizer = AutoTokenizer.from_pretrained(model_name)
gen_model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def clean_query(x):
    return [q[4:].strip() for q in x]

def query_generator(titles, prompt, gen_config, gen_model=gen_model, tokenizer=tokenizer, device=device):
    prompts = [prompt + f'{title}\nЗапросы:\n' for title in titles]
    inputs = tokenizer(
        prompts,
        return_tensors='pt',
        padding=True,
        truncation=True
    ).to(device)
    outputs = gen_model.generate(**inputs, **gen_config)
    gen_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    results = [text.split('Запросы:')[-1] for text in gen_texts]
    results = [regex.findall(r'\n[12]\..*', text) for text in results]

    queries = [clean_query(text) for text in results]

    return queries

In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/OTUS_project/prompt.txt'
with open(path, 'r', encoding='UTF-8') as f:
    prompt = f.read()

title = 'Новый гидрогель позволит хранить лекарства вне холодильника на протяжении нескольких недель'
gen_config = {
    'max_new_tokens': 30,
    'num_beams': 4,
    'early_stopping': True,
    'do_sample': False
}

x = query_generator([title], prompt, gen_config=gen_config)
print(x)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


[['достижения в области гидрогелей', 'хранение лекарств без холодильника']]


In [5]:
path = '/content/drive/MyDrive/Colab Notebooks/OTUS_project/articles_links.json'
all_data = pd.read_json(path)
df = all_data[['title', 'link', 'annot', 'tags']]
df

,title,link,annot,tags
0,"Ученые обнаружили еще одну причину, по которой...",https://naked-science.ru/article/biology/uchen...,Голые землекопы живут почти 40 лет — невероятн...,"[Биология, голые землекопы, ДНК, долголетие]"
1,"Бактерии создали из себя батарейку, чтобы разр...",https://naked-science.ru/article/biology/bacte...,Они использовали содержащееся в почве железо. ...,"[Биология, аккумулятор, бактерии, биопленки]"
2,Бактерии и добавки пероксида кальция сделают д...,https://naked-science.ru/article/column/bakter...,Ученые РГУ нефти и газа (НИУ) имени И.М. Губки...,"[Губкинский университет, арктика, бактерии, ка..."
3,Российские физики предложили искать темную мат...,https://naked-science.ru/article/column/temnuy...,Большой коллектив российских ученых из ведущих...,"[ФизТех, вселенная, квантование гравитации, те..."
4,Белорусские ученые вырастили кожу из грибов и ...,https://naked-science.ru/article/column/beloru...,Специалисты UST Inc. разработали технологию по...,"[Unitsky String Technologies Inc., биоматериал..."
...,...,...,...,...
8267,В Испании нашли свой Стоунхендж,https://naked-science.ru/article/anthropology/...,"На поле, предназначенном для выращивания авока...","[Антропология, археология, древние люди, Испания]"
8268,Астрономы оценили максимально возможное число ...,https://naked-science.ru/article/astronomy/mak...,"Ученые подсчитали, что без больших изменений о...","[Астрономия, естественный спутник, Луна, спутн..."
8269,Рецепт сознания: могут ли люди и роботы мечтат...,https://naked-science.ru/article/hi-tech/retse...,В XXI веке вопрос о природе сознания станет од...,"[Технологии, искусственный интеллект, машинное..."
8270,У берегов Западной Африки нашли древний ударны...,https://naked-science.ru/article/geology/udarn...,"Ученые допускают, что 66 миллионов лет назад с...","[Геология, астероид, гибель динозавров, диноза..."


In [6]:
len(df['title']), len(set(df['title']))

(8272, 8272)

In [ ]:
class TitlesDataset(Dataset):
  def __init__(self, ids, titles):
    self.ids = ids
    self.titles = titles

  def __len__(self):
    return len(self.ids)

  def __getitem__(self, idx):
    return int(self.ids[idx]), self.titles[idx]

In [ ]:
batch_size = 16
titles_loader = TitlesDataset(ids=df.index, titles=df['title'].values)
titles_loader = DataLoader(titles_loader, batch_size=batch_size, shuffle=False)
path = '/content/drive/MyDrive/Colab Notebooks/OTUS_project/req_from_titles.json'

errors = []
results = []

for i, (idx, titles) in tqdm(enumerate(titles_loader)):
  try:
    some_queries = query_generator(titles, prompt, gen_config=gen_config)
    for req, j, head in zip(some_queries, idx.tolist(), titles):
      results.append({'requests': req, 'id': j, 'title': head})
  except:
    errors.append(titles)

  if i % 10 == 0:
    with open(path, 'w', encoding='UTF-8') as f:
      json.dump(results, f, ensure_ascii=False, indent=4)


with open(path, 'w', encoding='UTF-8') as f:
      json.dump(results, f, ensure_ascii=False, indent=4)

0it [00:00, ?it/s]

In [ ]:
errors

[]

In [79]:
path = '/content/drive/MyDrive/Colab Notebooks/OTUS_project/req_from_titles.json'
req_from_titles = pd.read_json(path)
req_from_titles = req_from_titles[req_from_titles['requests'].map(len) == 2]

first_df = req_from_titles[['requests', 'id']].explode('requests', ignore_index=True).rename(columns={'requests': 'request'})
first_df['id'] = first_df['id'].map(lambda x: [x])
first_df

,request,id
0,почему голые землекопы живут так долго,[0]
1,открытия в изучении голых землекопов,[0]
2,статьи о бактериях и антибиотиках,[1]
3,бактериальные батареи,[1]
4,экологичные методы добычи нефти в Арктике,[2]
...,...,...
16339,мечты у людей и роботов,[8269]
16340,древние ударные кратеры,[8270]
16341,кратеры на берегах Западной Африки,[8270]
16342,статьи про углеводороды,[8271]


In [75]:
tags_df = df['tags'].explode().reset_index()
tags_list = list(tags_df['tags'].unique())
tags_lower = [s.lower() for s in tags_list]

def process_tag(s, tags_list=tags_list, tags_lower=tags_lower):
  pos = tags_lower.index(s.lower())
  return tags_list[pos]

tags_df['tags'] = tags_df['tags'].map(process_tag)
tags_df = tags_df.groupby('tags', as_index=False).agg({'index': list})

second_df = tags_df[tags_df['index'].map(len).between(2, 10, inclusive='neither')]
second_df = second_df.rename(columns={'tags': 'request', 'index': 'id'}).reset_index(drop=True)
second_df

,request,id
0,2D-материалы,"[1498, 4049, 4812]"
1,3D,"[3385, 4500, 4640]"
2,3D-моделирование,"[1579, 3335, 4831, 7493]"
3,3D-модель,"[1697, 1844, 3076, 6546, 6927]"
4,5G,"[712, 5437, 5884, 8019]"
...,...,...
1648,электроника,"[3177, 3873, 6023, 8123]"
1649,эмбриональное развитие,"[6392, 6895, 7748]"
1650,эпидемии,"[5118, 7153, 7775]"
1651,ядерная медицина,"[4793, 4794, 4911]"


In [83]:
requests_df = pd.concat([first_df, second_df], ignore_index=True)
requests_df['n'] = requests_df['id'].map(len)

path = '/content/drive/MyDrive/Colab Notebooks/OTUS_project/sinthetic_reqs.csv'
requests_df.to_csv(path)
requests_df

,request,id,n
0,почему голые землекопы живут так долго,[0],1
1,открытия в изучении голых землекопов,[0],1
2,статьи о бактериях и антибиотиках,[1],1
3,бактериальные батареи,[1],1
4,экологичные методы добычи нефти в Арктике,[2],1
...,...,...,...
17992,электроника,"[3177, 3873, 6023, 8123]",4
17993,эмбриональное развитие,"[6392, 6895, 7748]",3
17994,эпидемии,"[5118, 7153, 7775]",3
17995,ядерная медицина,"[4793, 4794, 4911]",3
